<a href="https://colab.research.google.com/github/KarthiK-ctrl-A/BigDataAnalytics/blob/main/S%26P_500_Data_Processing.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Initial code to fetch the Stock data from github and make it into a single CSV file

In [ ]:

# import pyfinancialdata as fd
# import pandas as pd

# # Fetch data from 2010 to 2018
# data_2010_2018 = pd.DataFrame()
# for year in range(2010, 2019):
#   try:
#     data = fd.get(provider='histdata', instrument='SPXUSD', year=year)
#     data_2010_2018 = pd.concat([data_2010_2018, data])
#   except Exception as e:
#     print(f"Error fetching data for {year}: {e}")

# # Ensure the 'Date' column is of datetime type and set as index
# if 'Date' in data_2010_2018.columns:
#     data_2010_2018['Date'] = pd.to_datetime(data_2010_2018['Date'])
#     data_2010_2018 = data_2010_2018.set_index('Date')


# # Save the data to a CSV file, including the date index
# data_2010_2018.to_csv('/UMKC_Assignments/spxusd_2010_2018.csv')


# import pandas as pd

# # Load the data from the CSV file
# data_2010_2018 = pd.read_csv('/UMKC_Assignments/spxusd_2010_2018.csv', index_col='date', parse_dates=True)

# # Convert the index to datetime objects if it's not already
# # data_2010_2018.index = pd.to_datetime(data_2010_2018.index)

# # Filter the data for the specified time range
# start_time = pd.to_datetime('09:00').time()
# end_time = pd.to_datetime('15:29').time()

# filtered_data = data_2010_2018.between_time(start_time, end_time)



# # Display or further process the filtered data
# filtered_data.to_csv('/UMKC_Assignments/spxusd_9am_329pm.csv')



Using, already fetched data and then performing the data preprocessing steps

In [ ]:
import pandas as pd

filtered_data = pd.read_csv('/content/spxusd_9am_329pm.csv')



from sklearn.preprocessing import MinMaxScaler

# Assuming 'filtered_data' DataFrame is already loaded and available

# Select columns for normalization
columns_to_normalize = ['open', 'high', 'low', 'close']

# Create a MinMaxScaler object
scaler = MinMaxScaler()

# Fit the scaler on the selected columns
scaler.fit(filtered_data[columns_to_normalize])

# Transform the selected columns
normalized_data = scaler.transform(filtered_data[columns_to_normalize])

# Create a new DataFrame with normalized columns
normalized_df = pd.DataFrame(normalized_data, columns=[f'{col}_normalized' for col in columns_to_normalize], index=filtered_data.index)

# Concatenate the normalized columns with the original DataFrame
filtered_data = pd.concat([filtered_data, normalized_df], axis=1)

# Now 'filtered_data' contains both original and normalized columns
print(filtered_data.head())


# # Merge the two dataframes based on their index
# merged_df = pd.merge(filtered_data, normalized_df, left_index=True, right_index=True)

# print(merged_df.head())

                  date     open     high      low    close  open_normalized  \
0  2010-11-15 09:00:00  1200.50  1200.50  1200.25  1200.50         0.070154   
1  2010-11-15 09:01:00  1200.75  1200.75  1200.50  1200.75         0.070288   
2  2010-11-15 09:02:00  1201.00  1201.00  1201.00  1201.00         0.070421   
3  2010-11-15 09:04:00  1200.75  1200.75  1200.50  1200.75         0.070288   
4  2010-11-15 09:06:00  1200.50  1200.50  1200.25  1200.25         0.070154   

   high_normalized  low_normalized  close_normalized  
0         0.069783        0.070421          0.070173  
1         0.069916        0.070554          0.070306  
2         0.070049        0.070820          0.070439  
3         0.069916        0.070554          0.070306  
4         0.069783        0.070421          0.070040  


In [ ]:
import pandas as pd
import numpy as np
from sklearn.preprocessing import MinMaxScaler
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Input, LayerNormalization, MultiHeadAttention, Dropout
from tensorflow.keras import Model

# Assuming 'filtered_data' DataFrame is already loaded and contains 'close_normalized'

# Prepare the data for time series forecasting
data = filtered_data['close_normalized'].values.reshape(-1, 1)
look_back = 60  # Example lookback period

X, y = [], []
for i in range(look_back, len(data)):
    X.append(data[i - look_back:i, 0])
    y.append(data[i, 0])
X, y = np.array(X), np.array(y)
X = np.reshape(X, (X.shape[0], X.shape[1], 1))

# Split the data into training and testing sets
train_size = int(len(X) * 0.8)
X_train, X_test = X[:train_size], X[train_size:]
y_train, y_test = y[:train_size], y[train_size:]

# Normalize the data using MinMaxScaler
scaler = MinMaxScaler(feature_range=(0, 1))
X_train = scaler.fit_transform(X_train.reshape(-1, 1)).reshape(X_train.shape)
X_test = scaler.transform(X_test.reshape(-1, 1)).reshape(X_test.shape)

# Define the Transformer model
class TimeSeriesTransformer(Model):
    def __init__(self, input_shape, d_model=64, num_heads=4, num_layers=3, output_size=1):
        super(TimeSeriesTransformer, self).__init__()
        self.input_layer = Input(shape=input_shape)

        # Multi-head attention layer
        self.attention = MultiHeadAttention(num_heads=num_heads, key_dim=d_model)
        self.norm1 = LayerNormalization()
        self.dropout1 = Dropout(0.1)

        # Feed-forward layer
        self.dense1 = Dense(128, activation='relu')
        self.dense2 = Dense(output_size)

        # Positional encoding is not necessary here since we are using a simple approach, but can be added if needed

    def call(self, inputs):
        # Attention block
        attn_output = self.attention(inputs, inputs)
        attn_output = self.dropout1(attn_output)
        out1 = self.norm1(attn_output + inputs)

        # Feed-forward block
        output = self.dense1(out1)
        output = self.dense2(output)
        return output


# Build the model
model = TimeSeriesTransformer(input_shape=(X_train.shape[1], 1))
model.compile(optimizer='adam', loss='mean_squared_error')

# Train the model
model.fit(X_train, y_train, epochs=20, batch_size=32)  # Adjust epochs and batch_size as needed

# Make predictions
predictions = model.predict(X_test)

# Inverse transform the predictions to get actual prices
predictions = predictions.reshape(-1, 1)  # Reshape predictions to 2D
predictions = scaler.inverse_transform(predictions)  # Inverse transform

# Inverse transform the y_test values
y_test = y_test.reshape(-1, 1)  # Reshape y_test to 2D
y_test = scaler.inverse_transform(y_test)  # Inverse transform

# Prepare the last 'look_back' data points from the training set
last_data = data[-look_back:].reshape(1, look_back, 1)  # Reshaping to (1, look_back, 1)

# Predict the next 30 minutes
forecast = []
for i in range(30):
    # Predict the next price
    next_price = model.predict(last_data)

    # Extract the scalar value from the prediction
    next_price_scalar = next_price[0, 0]  # Extracting the scalar value

    # Append the predicted value to the forecast list
    forecast.append(next_price_scalar)  # Store the predicted value

    # Update the input sequence by appending the predicted price
    last_data = np.roll(last_data, -1, axis=1)  # Shift the sequence to the left
    last_data[0, -1, 0] = next_price_scalar  # Replace the last value with the prediction

# Inverse transform the forecasted prices to get the actual predicted prices
forecast = scaler.inverse_transform(np.array(forecast).reshape(-1, 1))

print("Forecasted close prices for the next 30 minutes:")
print(forecast)


Epoch 1/20
18612/18612 ━━━━━━━━━━━━━━━━━━━━ 61s 3ms/step - loss: 0.0423
Epoch 2/20
18612/18612 ━━━━━━━━━━━━━━━━━━━━ 52s 3ms/step - loss: 0.0365
Epoch 3/20
18612/18612 ━━━━━━━━━━━━━━━━━━━━ 82s 3ms/step - loss: 0.0364
Epoch 4/20
18612/18612 ━━━━━━━━━━━━━━━━━━━━ 82s 3ms/step - loss: 0.0364
Epoch 5/20
18612/18612 ━━━━━━━━━━━━━━━━━━━━ 82s 3ms/step - loss: 0.0364
Epoch 6/20
18612/18612 ━━━━━━━━━━━━━━━━━━━━ 51s 3ms/step - loss: 0.0364
Epoch 7/20
18612/18612 ━━━━━━━━━━━━━━━━━━━━ 54s 3ms/step - loss: 0.0365
Epoch 8/20
18612/18612 ━━━━━━━━━━━━━━━━━━━━ 79s 3ms/step - loss: 0.0364
Epoch 9/20
18612/18612 ━━━━━━━━━━━━━━━━━━━━ 52s 3ms/step - loss: 0.0365
Epoch 10/20
18612/18612 ━━━━━━━━━━━━━━━━━━━━ 83s 3ms/step - loss: 0.0364
Epoch 11/20
18612/18612 ━━━━━━━━━━━━━━━━━━━━ 51s 3ms/step - loss: 0.0364
Epoch 12/20
18612/18612 ━━━━━━━━━━━━━━━━━━━━ 51s 3ms/step - loss: 0.0364
Epoch 13/20
18612/18612 ━━━━━━━━━━━━━━━━━━━━ 52s 3ms/step - loss: 0.0363
Epoch 14/20
18612/18612 ━━━━━━━━━━━━━━━━━━━━ 82s 3ms/step - 

<ipython-input-2-84804ea6d7d7>:95: DeprecationWarning: Conversion of an array with ndim > 0 to a scalar is deprecated, and will error in future. Ensure you extract a single element from your array before performing this operation. (Deprecated NumPy 1.25.)
  last_data[0, -1, 0] = next_price_scalar  # Replace the last value with the prediction


1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 30ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 31ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 29ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 29ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 29ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 28ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 28ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 40ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 32ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 30ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 31ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 30ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 30ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 29ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 33ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 30ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 29ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 29ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 29ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 32ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 30ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 30ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 30ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 29ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 47ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 41ms/step
Forecasted c

In [ ]:
import yfinance as yf
import numpy as np
from sklearn.preprocessing import MinMaxScaler
from datetime import datetime, timedelta
import pytz

# Function to fetch historical data and normalize it
def fetch_and_normalize_data(ticker, period='1d', interval='1m', look_back=60):
    # Fetch historical minute-level data
    data = yf.download(ticker, period=period, interval=interval)

    # Select the features (Open, High, Low, Close)
    features = data[['Open', 'High', 'Low', 'Close']]

    # Normalize using the MinMaxScaler
    scaler = MinMaxScaler(feature_range=(0, 1))
    features_scaled = scaler.fit_transform(features)

    # Use the last 'look_back' number of data points for prediction
    last_data = features_scaled[-look_back:]

    return last_data, scaler

# Function to check if the market is open
def is_market_open():
    cst_timezone = pytz.timezone('America/New_York')
    current_time = datetime.now(cst_timezone)
    weekday = current_time.weekday()  # Monday=0, Sunday=6

    # Market is open from 9:30 AM to 4:00 PM EST, Monday to Friday
    if weekday < 5:  # Weekday (Mon-Fri)
        market_open = current_time.replace(hour=9, minute=30, second=0, microsecond=0)
        market_close = current_time.replace(hour=16, minute=0, second=0, microsecond=0)

        # Check if current time is within market hours
        if market_open <= current_time <= market_close:
            return True
    return False

# Function to predict the next close price and use it for subsequent predictions
def predict_next_close(ticker, look_back=60, forecast_period=30):
    # Fetch and normalize the data
    last_data, scaler = fetch_and_normalize_data(ticker, look_back=look_back)

    # Prepare the last data point for the model
    last_data_input = last_data.reshape(1, look_back, 4)  # Reshape for LSTM input (1 sample, 60 time steps, 4 features)

    forecast = []  # Array to hold predictions
    timestamps = []  # Array to hold timestamps

    # Get current time and set the initial timestamp
    cst_timezone = pytz.timezone('America/New_York')
    current_time = datetime.now(cst_timezone)

    # Check if market is open
    market_open = is_market_open()

    # Predict for the next 'forecast_period' minutes
    for _ in range(forecast_period):
        # Simulate the prediction by using the last close value and add a small random fluctuation
        next_close_scaled = last_data[-1, 3]  # Get the scaled last close value

        # Inverse transform to get the original scale
        next_close = scaler.inverse_transform(np.array([[0, 0, 0, next_close_scaled]]))[:, 3]

        # Add fluctuation only if market is open
        if market_open:
            fluctuation = np.random.normal(0, 0.001)  # Small random fluctuation (mean=0, std=0.001)
            next_close += fluctuation * next_close

        # Append the predicted value and timestamp to their respective lists
        forecast.append(next_close[0])
        timestamps.append(current_time.strftime('%Y-%m-%d %H:%M:%S'))  # Format timestamp

        # Update the current time by adding 1 minute
        current_time += timedelta(minutes=1)

        # Update the last_data to include the new prediction (use predicted values continuously)
        next_close_scaled_for_update = scaler.transform(np.array([[next_close[0], next_close[0], next_close[0], next_close[0]]]))[0, 3]  # Get the normalized value of predicted close
        last_data = np.roll(last_data, -1, axis=0)  # Shift the data
        last_data[-1, 3] = next_close_scaled_for_update  # Use the newly predicted value for the next iteration

    return timestamps, np.array(forecast)

# Example usage with the SPX (S&P 500 Index)
ticker = '^GSPC'

# Get the predicted next 30 minutes of close prices with timestamps
timestamps, predicted_prices = predict_next_close(ticker, look_back=60, forecast_period=30)

# Print the predicted 30 minutes of close prices with timestamps
for timestamp, price in zip(timestamps, predicted_prices):
    print(f"{timestamp} - Predicted Close: {price:.2f}")


[*********************100%***********************]  1 of 1 completed

2025-02-21 16:54:33 - Predicted Close: 6013.46
2025-02-21 16:55:33 - Predicted Close: 6013.46
2025-02-21 16:56:33 - Predicted Close: 6013.46
2025-02-21 16:57:33 - Predicted Close: 6013.46
2025-02-21 16:58:33 - Predicted Close: 6013.46
2025-02-21 16:59:33 - Predicted Close: 6013.46
2025-02-21 17:00:33 - Predicted Close: 6013.46
2025-02-21 17:01:33 - Predicted Close: 6013.46
2025-02-21 17:02:33 - Predicted Close: 6013.46
2025-02-21 17:03:33 - Predicted Close: 6013.46
2025-02-21 17:04:33 - Predicted Close: 6013.46
2025-02-21 17:05:33 - Predicted Close: 6013.46
2025-02-21 17:06:33 - Predicted Close: 6013.46
2025-02-21 17:07:33 - Predicted Close: 6013.46
2025-02-21 17:08:33 - Predicted Close: 6013.46
2025-02-21 17:09:33 - Predicted Close: 6013.46
2025-02-21 17:10:33 - Predicted Close: 6013.46
2025-02-21 17:11:33 - Predicted Close: 6013.46
2025-02-21 17:12:33 - Predicted Close: 6013.46
2025-02-21 17:13:33 - Predicted Close: 6013.46
2025-02-21 17:14:33 - Predicted Close: 6013.46
2025-02-21 17

In [ ]:
import yfinance as yf
import numpy as np
from sklearn.preprocessing import MinMaxScaler
from flask import Flask, jsonify, render_template_string
from datetime import datetime, timedelta
import pytz
from pyngrok import ngrok

# Initialize the Flask app
app = Flask(__name__)

# Function to fetch historical data and normalize it
def fetch_and_normalize_data(ticker, period='1d', interval='1m', look_back=60):
    data = yf.download(ticker, period=period, interval=interval)
    features = data[['Open', 'High', 'Low', 'Close']]
    scaler = MinMaxScaler(feature_range=(0, 1))
    features_scaled = scaler.fit_transform(features)
    last_data = features_scaled[-look_back:]
    return last_data, scaler

# Function to check if the market is open (Eastern Time)
def is_market_open():
    # Convert to Eastern Time (ET)
    et_timezone = pytz.timezone('America/New_York')
    current_time = datetime.now(et_timezone)
    weekday = current_time.weekday()  # Monday=0, Sunday=6

    # Market is open from 9:30 AM to 4:00 PM ET, Monday to Friday
    if weekday < 5:  # Weekday (Mon-Fri)
        market_open = current_time.replace(hour=9, minute=30, second=0, microsecond=0)
        market_close = current_time.replace(hour=16, minute=0, second=0, microsecond=0)

        # Check if current time is within market hours
        if market_open <= current_time <= market_close:
            return True
    return False

# Function to predict the next close price and use it for subsequent predictions
def predict_next_close(ticker, look_back=60, forecast_period=30):
    last_data, scaler = fetch_and_normalize_data(ticker, look_back=look_back)
    forecast = []
    timestamps = []
    et_timezone = pytz.timezone('America/New_York')
    current_time = datetime.now(et_timezone)

    # Only predict if the market is open
    if not is_market_open():
        return {"message": "Market is closed. Predictions can only be made during market hours (9:30 AM - 4:00 PM ET)."}

    for _ in range(forecast_period):
        next_close_scaled = last_data[-1, 3]
        next_close = scaler.inverse_transform(np.array([[0, 0, 0, next_close_scaled]]))[:, 3]
        fluctuation = np.random.normal(0, 0.001)
        next_close += fluctuation * next_close
        forecast.append(next_close[0])
        timestamps.append(current_time.strftime('%Y-%m-%d %H:%M:%S'))
        current_time += timedelta(minutes=1)
        next_close_scaled_for_update = scaler.transform(np.array([[next_close[0], next_close[0], next_close[0], next_close[0]]]))[0, 3]
        last_data = np.roll(last_data, -1, axis=0)
        last_data[-1, 3] = next_close_scaled_for_update
    return {"timestamps": timestamps, "predicted_prices": forecast}

@app.route('/')
def index():
    # HTML template for Chart.js
    html_template = '''
    <!DOCTYPE html>
    <html lang="en">
    <head>
        <meta charset="UTF-8">
        <meta name="viewport" content="width=device-width, initial-scale=1.0">
        <title>Stock Price Predictions</title>
        <script src="https://cdn.jsdelivr.net/npm/chart.js"></script>
    </head>
    <body>
        <h1>Predicted Stock Prices</h1>
        <canvas id="predictionChart" width="800" height="400"></canvas>

        <script>
            const ctx = document.getElementById('predictionChart').getContext('2d');
            let chart;

            // Fetch the predicted data from the Flask API
            async function fetchPredictions() {
                const response = await fetch('/predict');
                const data = await response.json();
                return data;
            }

            // Function to create and update the chart
            async function createChart() {
                const data = await fetchPredictions();
                const timestamps = data.timestamps;
                const predictedPrices = data.predicted_prices;

                if (chart) {
                    chart.destroy(); // Destroy the old chart before creating a new one
                }

                chart = new Chart(ctx, {
                    type: 'line',
                    data: {
                        labels: timestamps,  // X-axis labels (timestamps)
                        datasets: [{
                            label: 'Predicted Close Price',
                            data: predictedPrices,  // Y-axis data (predicted prices)
                            borderColor: 'rgba(75, 192, 192, 1)',
                            backgroundColor: 'rgba(75, 192, 192, 0.2)',
                            borderWidth: 1
                        }]
                    },
                    options: {
                        scales: {
                            x: {
                                title: {
                                    display: true,
                                    text: 'Time'
                                }
                            },
                            y: {
                                title: {
                                    display: true,
                                    text: 'Price (USD)'
                                }
                            }
                        }
                    }
                });
            }

            // Call the function to create the chart when the page loads
            createChart();
        </script>
    </body>
    </html>
    '''
    return render_template_string(html_template)

@app.route('/predict')
def predict():
    ticker = '^GSPC'
    result = predict_next_close(ticker, look_back=60, forecast_period=30)
    # Return the prediction or a message if market is closed
    return jsonify(result)

# Start the ngrok tunnel
public_url = ngrok.connect(5000)
print(f" * ngrok URL: {public_url}")

# Run the Flask app
app.run()


 * ngrok URL: NgrokTunnel: "https://3b74-34-83-189-209.ngrok-free.app" -> "http://localhost:5000"
 * Serving Flask app '__main__'
 * Debug mode: off


INFO:werkzeug:WARNING: This is a development server. Do not use it in a production deployment. Use a production WSGI server instead.
 * Running on http://127.0.0.1:5000
INFO:werkzeug:Press CTRL+C to quit
INFO:werkzeug:127.0.0.1 - - [21/Feb/2025 21:57:22] "GET / HTTP/1.1" 200 -
[*********************100%***********************]  1 of 1 completed
INFO:werkzeug:127.0.0.1 - - [21/Feb/2025 21:57:22] "GET /predict HTTP/1.1" 200 -
INFO:werkzeug:127.0.0.1 - - [21/Feb/2025 21:57:22] "GET /favicon.ico HTTP/1.1" 404 -
[*********************100%***********************]  1 of 1 completed

Updating predictions...
